In [14]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
from statsmodels.formula.api import ols
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import KFold, cross_val_score
from sklearn.metrics import  mean_absolute_error
from sklearn.model_selection import train_test_split

In [9]:
sample = pd.read_csv("data_for_model.csv")


In [10]:
X = sample[['Diện Tích', 'Triệu/m2', 'Số Phòng Ngủ', 'Số Nhà Vệ Sinh',
       'Giá Đề Xuất', 'Giá Quận', 'Giá Đường']]
y = sample['Giá']

# Thêm một hằng số (constant) vào mô hình
X = sm.add_constant(X)

# Xây dựng mô hình hồi quy OLS
model = sm.OLS(y, X).fit()

# Hiển thị kết quả mô hình
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:                    Giá   R-squared:                       0.657
Model:                            OLS   Adj. R-squared:                  0.657
Method:                 Least Squares   F-statistic:                     1373.
Date:                Thu, 26 Dec 2024   Prob (F-statistic):               0.00
Time:                        20:21:35   Log-Likelihood:                -20100.
No. Observations:                5015   AIC:                         4.022e+04
Df Residuals:                    5007   BIC:                         4.027e+04
Df Model:                           7                                         
Covariance Type:            nonrobust                                         
                     coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------
const             -8.6472      0.651    -13.

In [11]:
# Tách X (đặc trưng) và y (mục tiêu)
X = sample.drop(columns=["Giá"])
y = sample["Giá"]  # Cột mục tiêu

# Chia dữ liệu thành 80% train và 20% test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

def OLS(X_train, y_train):

    # Khởi tạo và huấn luyện mô hình hồi quy tuyến tính
    model = LinearRegression()
    model.fit(X_train, y_train)

    # Trả về mô hình
    return model

#2b.
def find_best_feature(train, features, random_state, n_splits):
    """
    Đánh giá các đặc trưng bằng cách sử dụng cross-validation để tìm đặc trưng tốt nhất dựa trên MAE.

    Parameters:
    - train: DataFrame chứa dữ liệu huấn luyện.
    - features: Danh sách các tên cột đặc trưng cần thử nghiệm.
    - n_splits: Số lượng folds trong cross-validation.
    - random_state: Tham số ngẫu nhiên cho việc xáo trộn dữ liệu.

    Returns:
    - best_feature: Tên đặc trưng tốt nhất.
    """

    # Xáo trộn dữ liệu
    train_data_shuffled = train.sample(frac=1, random_state=random_state).reset_index(drop=True)

    # Khởi tạo mô hình
    model = LinearRegression()

    # Khởi tạo Cross-Validation
    kf = KFold(n_splits=n_splits, shuffle=False)
    best_mae = float('inf')
    best_feature = None

    # Thử nghiệm từng đặc trưng một
    for feature in features:
        X = train_data_shuffled[[feature]]
        y = train_data_shuffled['Giá']
        
        # Đánh giá mô hình sử dụng Cross-Validation với MAE
        mae_scores = -cross_val_score(model, X, y, cv=kf, scoring='neg_mean_absolute_error')
        mean_mae = mae_scores.mean()
        
        print(f"Feature: {feature}, Cross-Validation MAE: {mean_mae:.3f}")
        
        # Cập nhật nếu có đặc trưng tốt hơn
        if mean_mae < best_mae:
            best_mae = mean_mae
            best_feature = feature

    # Trả về thuộc tính tốt nhất
    return best_feature


# Gọi hàm OLS để trả về mô hình được xây dựng bằng thư viện và được huấn luyện bằng tập train 
model = OLS(X_train, y_train)

# Lấy các tham số trả về từ mô hình
coefficients = model.coef_
intercept = model.intercept_

# Công thức hồi quy 
regression_formula = f"Giá = {intercept:.3f} + " + " + ".join([f"({coeff:.3f})*{feature}" for coeff, feature in zip(coefficients, X_train.columns)])
regression_formula

# Chạy xong khúc này ông chạy khúc dưới gọi hàm MAE để cho ra kết quả nha, còn nếu muốn chạy mô hình khác thì là cell ngay dưới luôn chạy rồi chạy MAE lại.

'Giá = -7.560 + (-0.000)*Unnamed: 0 + (0.047)*Diện Tích + (0.137)*Triệu/m2 + (0.170)*Số Phòng Ngủ + (0.158)*Số Nhà Vệ Sinh + (-0.000)*Giá Đề Xuất + (-0.618)*Giá Quận + (0.338)*Giá Đường'

In [12]:
# Hoặc mô hình RandomForestRegressorRandomForestRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score

# Tạo mô hình
model = RandomForestRegressor(n_estimators=100, random_state=42)

# Huấn luyện
model.fit(X_train, y_train)

RandomForestRegressor(random_state=42)

In [15]:
# Gọi hàm MAE (tự cài đặt hoặc từ thư viện) trên tập kiểm tra

# Gọi dự đoán treen tập test
y_pred = model.predict(X_test)
# Tính toán độ lỗi tuyệt đối trung bình (MAE)
mae = mean_absolute_error(y_test, y_pred)
print(f"MAE: {mae:.3f}")

# Lấy một chỉ số ngẫu nhiên từ tập kiểm thử
random_index = np.random.randint(0, len(X_test))

# Lấy giá trị thử nghiệm từ X_test
X_sample = X_test.iloc[random_index].values.reshape(1, -1)

# Hiển thị giá trị thử nghiệm
print("Giá trị thử nghiệm (X_sample):")
print(X_sample)

# Dự đoán giá trị y tương ứng cho X_sample
y_pred_sample = model.predict(X_sample)

# Hiển thị giá trị dự đoán và giá trị thực tế từ y_test
print(f"\nGiá trị thực tế (y_test[{random_index}]): {y_test.iloc[random_index]}")
print(f"Giá trị dự đoán: {y_pred_sample[0]}")

MAE: 0.625
Giá trị thử nghiệm (X_sample):
[[5.711e+03 9.700e+01 1.250e+02 3.000e+00 2.000e+00 1.612e+05 6.950e+00
  8.050e+00]]

Giá trị thực tế (y_test[942]): 12.125
Giá trị dự đoán: 12.092799999999997


c:\ProgramData\anaconda3\Lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(
